<a href="https://colab.research.google.com/github/veronica-araoz/veronica-araoz_clustering/blob/main/Polarizacion_en_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# 🔹 Análisis de polaridad con RoBERTuito (pysentimiento)
#    + Probabilidades y confianza
# ============================================================

# 1️⃣ Instalar dependencias
!pip install pysentimiento pandas openpyxl

# 2️⃣ Importar librerías
import pandas as pd
from google.colab import drive
from pysentimiento import create_analyzer

# 3️⃣ Conectar con Google Drive
drive.mount('/content/drive')

# 4️⃣ Ruta del archivo en tu Drive
#    Podés simplemente poner:
#    /content/drive/MyDrive/QuinteroCalle.csv
ruta_archivo = "/content/drive/MyDrive/SCRIPT VERO/QuinteroCalle (1).csv"

print(f"📂 Cargando archivo: {ruta_archivo}")

# 5️⃣ Cargar el dataset (solo CSV en tu caso)
df = pd.read_csv(ruta_archivo)

print("✅ Dataset cargado correctamente. Columnas disponibles:")
print(df.columns.tolist())

# 6️⃣ Crear el analizador RoBERTuito
analyzer = create_analyzer(task="sentiment", lang="es")

# 7️⃣ Aplicar análisis con probabilidades
if 'rawContent' in df.columns:

    def analizar(texto):
        pred = analyzer.predict(str(texto))
        return pd.Series({
            'RoBERTuito': pred.output,
            'RoBERTuito_probas': pred.probas,
            'RoBERTuito_confianza': pred.probas[pred.output]
        })

    resultados = df['rawContent'].apply(analizar)

    # Unir las nuevas columnas al dataframe original
    df = pd.concat([df, resultados], axis=1)

    print("✅ Análisis completado. Columnas agregadas:")
    print("   - RoBERTuito")
    print("   - RoBERTuito_probas")
    print("   - RoBERTuito_confianza")

else:
    print("❌ No se encontró la columna 'rawContent'. Verificá el nombre en tu archivo.")

# 8️⃣ Guardar el dataset actualizado en Drive
ruta_salida = ruta_archivo.replace(".csv", "_RoBERTuito.xlsx")
df.to_excel(ruta_salida, index=False)

print(f"💾 Archivo con la clasificación guardado en:\n{ruta_salida}")

In [ ]:
# ============================================================
# 🔹 1. Importar librerías
# ============================================================
import pandas as pd

# ============================================================
# 🔹 2. Definir la ruta del archivo generado por tu script
# ============================================================
ruta = "/content/drive/MyDrive/SCRIPT VERO/QuinteroCalle (1)_RoBERTuito.xlsx"

# ============================================================
# 🔹 3. Cargar el archivo Excel
# ============================================================
df = pd.read_excel(ruta)

# Mostrar las columnas disponibles
print("📋 Columnas en el archivo:")
print(df.columns.tolist())

# ============================================================
# 🔹 4. Nombre de la columna con la clasificación (ajustalo si hace falta)
# ============================================================
col_sent = "RoBERTuito"   # Cambiar si tiene otro nombre

if col_sent not in df.columns:
    raise ValueError(f"La columna '{col_sent}' no existe en el archivo.")

# ============================================================
# 🔹 5. Resumen del análisis de sentimientos
# ============================================================
conteo = df[col_sent].value_counts()
print("\n📊 Distribución de sentimientos:")
print(conteo)

# Porcentajes
porcentaje = (conteo / len(df) * 100).round(2)
print("\n📈 Porcentajes:")
print(porcentaje)

# ============================================================
# 🔹 6. Ejemplos de tuits por categoría (si existe rawContent)
# ============================================================
if "rawContent" in df.columns:

    if "POS" in conteo.index:
        print("\n🟢 Ejemplos POSITIVOS:")
        print(df[df[col_sent] == "POS"]['rawContent'].sample(min(3, conteo["POS"]), random_state=1).to_list())

    if "NEG" in conteo.index:
        print("\n🔴 Ejemplos NEGATIVOS:")
        print(df[df[col_sent] == "NEG"]['rawContent'].sample(min(3, conteo["NEG"]), random_state=1).to_list())

    if "NEU" in conteo.index:
        print("\n⚪ Ejemplos NEUTROS:")
        print(df[df[col_sent] == "NEU"]['rawContent'].sample(min(3, conteo["NEU"]), random_state=1).to_list())

else:
    print("\n⚠️ El archivo no contiene la columna 'rawContent'.")


In [ ]:
import pandas as pd
import ast   # <-- necesario para convertir texto a diccionario

# ============================================================
# 1. Cargar el archivo
# ============================================================
ruta = "/content/drive/MyDrive/SCRIPT VERO/QuinteroCalle (1)_RoBERTuito.xlsx"
df = pd.read_excel(ruta)

# ============================================================
# 2. Convertir strings a diccionarios reales
# ============================================================
def convertir_a_dict(x):
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return None
    return None

df["RoBERTuito_probas"] = df["RoBERTuito_probas"].apply(convertir_a_dict)

# Verificar si alguna fila quedó sin convertir
if df["RoBERTuito_probas"].isnull().any():
    print("⚠️ Atención: algunas probabilidades no pudieron convertirse.")

# ============================================================
# 3. Extraer la confianza (máxima probabilidad)
# ============================================================
df["confianza"] = df["RoBERTuito_probas"].apply(lambda d: max(d.values()) if isinstance(d, dict) else None)

# ============================================================
# 4. Clasificar según nivel de duda
# ============================================================
def clasificar(conf):
    if conf is None:
        return "sin_datos"
    if conf < 0.33:
        return "practicamente_al_azar"
    elif conf < 0.40:
        return "sospechoso"
    elif conf < 0.60:
        return "ambiguedad_moderada"
    else:
        return "confiable"

df["nivel_duda"] = df["confianza"].apply(clasificar)

# ============================================================
# 5. Contar cuántos hay en cada categoría
# ============================================================
conteo = df["nivel_duda"].value_counts()

print("📊 Cantidad de tuits por nivel de duda:")
print(conteo)

print("\n📈 Porcentajes:")
print((conteo / len(df) * 100).round(2))


In [ ]:
# ============================================================
# 🛠️ Script optimizado para el archivo QuinteroCalleAzar.xlsx
#    (Asume: Librerías instaladas y Drive ya montado)
# ============================================================

# 1️⃣ Importar librerías (solo las necesarias para la lógica de este bloque)
import pandas as pd
from pysentimiento import create_analyzer

# 2️⃣ Ruta del nuevo archivo en tu Drive (¡Debe ser .xlsx o .csv!)
ruta_archivo = "/content/drive/MyDrive/SCRIPT VERO/QuinteroCalleAzar.xlsx"

print(f"📂 Cargando archivo: {ruta_archivo}")

# 3️⃣ Cargar el dataset (usando pd.read_excel para el formato asumido .xlsx)
# Si lo guardaste como .csv, usa pd.read_csv(ruta_archivo)
df = pd.read_excel(ruta_archivo)

print("✅ Dataset cargado correctamente.")
print(df.columns.tolist())

# 4️⃣ Crear/Recuperar el analizador RoBERTuito
analyzer = create_analyzer(task="sentiment", lang="es")
print("✅ Analizador RoBERTuito cargado.")

# 5️⃣ Aplicar análisis con probabilidades (Lógica del análisis)
if 'rawContent' in df.columns:

    def analizar(texto):
        pred = analyzer.predict(str(texto))
        return pd.Series({
            'RoBERTuito': pred.output,
            'RoBERTuito_probas': pred.probas,
            'RoBERTuito_confianza': pred.probas[pred.output]
        })

    resultados = df['rawContent'].apply(analizar)

    # Unir las nuevas columnas al dataframe original
    df = pd.concat([df, resultados], axis=1)

    print("✅ Análisis completado. Columnas agregadas:")
    print("   - RoBERTuito")

else:
    print("❌ No se encontró la columna 'rawContent'. Verificá el nombre.")

# 6️⃣ Guardar el dataset actualizado en Drive
# Adaptamos la ruta de salida para el nuevo archivo .xlsx
ruta_salida = ruta_archivo.replace(".xlsx", "ManualYRobertuito.xlsx")
df.to_excel(ruta_salida, index=False)

print(f"💾 Archivo con la clasificación guardado en:\n{ruta_salida}")

In [ ]:
import pandas as pd

# Cargar archivo
df = pd.read_excel("/content/drive/MyDrive/SCRIPT VERO/QuinteroCalleAzarManualYRobertuito.xlsx")

# Verificar que las columnas existan
print(df.columns)

# Filtrar coincidencias y no coincidencias
coinciden = df[df["Manual"] == df["RoBERTuito"]]
no_coinciden = df[df["Manual"] != df["RoBERTuito"]]

# Cantidades
n_total = len(df)
n_coinciden = len(coinciden)
n_no_coinciden = len(no_coinciden)

# Porcentaje
porcentaje = (n_coinciden / n_total) * 100

print("Total de filas:", n_total)
print("Coincidencias:", n_coinciden)
print("No coincidencias:", n_no_coinciden)
print(f"Porcentaje de coincidencia: {porcentaje:.2f}%")

# Cinco ejemplos de no coincidencias
print("\nEjemplos de no coincidencias:")
print(no_coinciden.head(5))


In [ ]:
# ============================================================
# 1) Montar Google Drive
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

print("✔ Drive montado correctamente.\n")


# ============================================================
# 2) Ingresar la ruta del archivo
#    (Pegá la ruta completa de tu archivo .xlsx o .csv en Drive)
# ============================================================

ruta = input("Ingresa la ruta completa del archivo en Drive: ")

print(f"📂 Cargando archivo desde:\n{ruta}\n")


# ============================================================
# 3) Cargar el dataset
# ============================================================

import pandas as pd

if ruta.endswith(".xlsx"):
    df = pd.read_excel(ruta)
elif ruta.endswith(".csv"):
    df = pd.read_csv(ruta)
else:
    raise ValueError("El archivo debe ser .xlsx o .csv")

print("✔ Dataset cargado. Columnas encontradas:")
print(df.columns.tolist())


# ============================================================
# 4) Validar columnas necesarias
# =
